# 1. Ifcopenshell & PythonOCC

In [8]:
import sys
sys.path.append('C:/Users/User/Anaconda3/Lib/site-packages')
sys.path

['',
 'c:\\python\\python35.zip',
 'c:\\python\\DLLs',
 'c:\\python\\lib',
 'c:\\python',
 'c:\\python\\lib\\site-packages',
 'c:\\python\\lib\\site-packages\\kenchi-0.6.0-py3.5.egg',
 'c:\\python\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\User\\.ipython',
 'C:/Users/User/Anaconda3/Lib/site-packages']

### 1. import packages - pythonOCC, ifcopenshell, etc.

In [9]:
# import pythonocc and other packages
import OCC.gp
import OCC.Geom
import OCC.Bnd
import OCC.BRepBndLib
import OCC.BRep
import OCC.BRepPrimAPI
import OCC.BRepAlgoAPI
import OCC.BRepBuilderAPI
import OCC.GProp
import OCC.BRepGProp
import OCC.TopoDS
import OCC.TopExp
import OCC.TopAbs
import ifcopenshell
import ifcopenshell.geom
import operator
import numpy as np
import pandas as pd
import datetime
import os
np.warnings.filterwarnings('ignore')

### 2. ifc객체를 ifc 파일로부터 추출하는 함수 get_features

In [4]:
def get_features(ifctype, path):
    # input (ifctype) : 추출하고 싶은 ifc객체 타입 e.g. ifcwallstandardcase, ifcdoor...
    # input (path) : ifc파일의 경로 
    # output : 추출한 ifc 객체들. pandas dataframe에 들어가있음
    
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_PYTHON_OPENCASCADE, True)
    settings.set(settings.USE_BREP_DATA, True)
    settings.set(settings.SEW_SHELLS, True)

    # settings.set(settings.USE_BREP_DATA,True)
    # settings.set(settings.SEW_SHELLS,True)
    # settings.set(settings.USE_WORLD_COORDS,True)

    def get_shape(elem):
        return ifcopenshell.geom.create_shape(settings, elem)

    def get_volume(s):
        props = OCC.GProp.GProp_GProps()
        OCC.BRepGProp.brepgprop_VolumeProperties(s.geometry, props)
        return props.Mass()

    def get_area(s):
        props = OCC.GProp.GProp_GProps()
        OCC.BRepGProp.brepgprop_SurfaceProperties(s.geometry, props)
        return props.Mass()

    def normalize(li):
        mean, std = np.mean(li), np.std(li)
        return map(lambda v: abs(v - mean) / std, li)

    def get_linear(s):
        props = OCC.GProp.GProp_GProps()
        OCC.BRepGProp.brepgprop_LinearProperties(s.geometry, props)
        return props.Mass()

    def get_global_id(elem):
        return elem[0]

    def get_gyration(s):
        props = OCC.GProp.GProp_GProps()
        OCC.BRepGProp.brepgprop_VolumeProperties(s.geometry, props)
        gyradius = props.RadiusOfGyration(OCC.gp.gp_Ax1(props.CentreOfMass(),
                                                        OCC.gp.gp_DZ()))
        return gyradius

    def get_boundingbox(shape, tol=1e-1, as_vec=True):
        bbox = OCC.Bnd.Bnd_Box()
        bbox.SetGap(tol)
        OCC.BRepBndLib.brepbndlib_Add(shape, bbox)
        xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
        vec = [OCC.gp.gp_Vec(xmin, ymin, zmin), OCC.gp.gp_Vec(xmax, ymax, zmax)]
        X = (vec[0].X() - vec[1].X()) * (-1)
        Y = (vec[0].Y() - vec[1].Y()) * (-1)
        Z = (vec[0].Z() - vec[1].Z()) * (-1)
        return (X, Y, Z)


    def get_type(elem):
        return (elem.is_a())

    def get_ax1(elem):  # area/volume
        return (get_area(elem) / get_volume(elem))

    def get_ax2(elem):  # gyration/volume
        return (get_gyration(elem) / get_volume(elem))

    def get_topods(elem):
        return elem[1]

    def get_X(elem):
        return elem[0]

    def get_Y(elem):
        return elem[1]

    def get_Z(elem):
        return elem[2]

    file = ifcopenshell.open(path)
    ifc = file.by_type(ifctype)
    shapes = list(map(get_shape, ifc))
    volumes = list(map(get_volume, shapes))
    areas = list(map(get_area, shapes))
    # linear = list(map(get_linear, shapes))
    global_ids = list(map(get_global_id, ifc))
    gyrations = list(map(get_gyration, shapes))
    types = list(map(get_type, ifc))
    ax1s = list(map(get_ax1, shapes))
    ax2s = list(map(get_ax2, shapes))
    topo = list(map(get_topods, shapes))
    bounding_value = list(map(get_boundingbox, topo))
    bat_ids = [str(i).split('=')[0] for i in ifc]
    X = list(map(get_X, bounding_value))
    Y = list(map(get_Y, bounding_value))
    Z = list(map(get_Z, bounding_value))

    data = {'areas': areas, 'volumes': volumes, 'gyrations': gyrations,
            'global_ids': global_ids, 'bat_ids':bat_ids, 'types': types, 'ax1s': ax1s, 'ax2s': ax2s,
            'X': X, 'Y': Y, 'Z': Z}

    frame = pd.DataFrame(data)
    return frame

### 3. 함수 사용 예시

In [15]:
# duplex 파일에서 ifcwindow 파일을 추출
duplex_window = get_features('ifcwindow', r"C:\Users\User\PycharmProjects\BIM\ifc\Duplex_A_20110907_optimized.ifc")

In [17]:
duplex_window.head(10)

,X,Y,Z,areas,ax1s,ax2s,bat_ids,global_ids,gyrations,types,volumes
0,0.617,3.000,2.610,23.751856,107.088435,4.579880,#226,1l0GAJtRTFv8$zmKJOH4pU,1.015802,IfcWindow,0.221797
1,0.617,3.000,2.610,23.751856,107.088435,4.579880,#227,1l0GAJtRTFv8$zmKJOH4$e,1.015802,IfcWindow,0.221797
2,1.019,0.617,0.959,4.249626,87.212998,6.789223,#228,1hOSvn6df7F8_7GcBWlS2V,0.330818,IfcWindow,0.048727
3,1.019,0.617,0.959,4.249626,87.212998,6.789223,#229,1hOSvn6df7F8_7GcBWlSga,0.330818,IfcWindow,0.048727
4,0.617,0.950,2.400,9.044896,91.664386,3.369757,#230,1hOSvn6df7F8_7GcBWlRRL,0.332508,IfcWindow,0.098674
5,0.617,0.950,2.400,9.044896,91.664386,3.369757,#231,1hOSvn6df7F8_7GcBWlRLx,0.332508,IfcWindow,0.098674
6,5.035,0.617,2.620,37.739076,111.891593,4.862685,#268,1hOSvn6df7F8_7GcBWlR72,1.640098,IfcWindow,0.337282
7,0.617,0.950,2.400,9.044896,91.664386,3.369757,#269,1l0GAJtRTFv8$zmKJOH4ZZ,0.332508,IfcWindow,0.098674
8,0.617,0.950,2.400,9.044896,91.664386,3.369757,#270,1l0GAJtRTFv8$zmKJOH4aS,0.332508,IfcWindow,0.098674
9,0.617,1.019,0.959,4.249626,87.212998,6.789223,#271,1l0GAJtRTFv8$zmKJOH4hv,0.330818,IfcWindow,0.048727


### 4. 하나의 ifc 파일에서 ifc 8개 타입 모두 추출하는 함수 get_building_ifc

In [1]:
def get_building_ifc(path):
    column = get_features('IfcColumn', path)
    beam = get_features('IfcBeam', path)
    slab = get_features('IfcSlab', path)
    wall = get_features('IfcWallStandardCase', path)
    covering = get_features('IfcCovering', path)
    door = get_features('IfcDoor', path)
    window = get_features('IfcWindow', path)
    railing = get_features('IfcRailing', path)
    df = pd.concat([column, beam, slab, wall, covering, door, window, railing])
    return df

### 5. 함수 사용 예시

In [11]:
# duplex 파일에서 ifcwindow 파일을 추출
duplex_all = get_building_ifc(r"C:\Users\User\PycharmProjects\BIM\ifc\Duplex_A_20110907_optimized.ifc")

In [12]:
duplex_all.head(10)

,X,Y,Z,areas,ax1s,ax2s,bat_ids,global_ids,gyrations,types,volumes
0,0.403000,7.62130,0.503,10.174007,177.246504,37.332488,#171,2OrWItJ6zAwBNp0OUxK$Dv,2.142897,IfcBeam,0.057400
1,6.381887,0.37800,0.607,9.171307,191.490330,37.269382,#172,2OrWItJ6zAwBNp0OUxK$Du,1.784993,IfcBeam,0.047894
2,0.378000,4.32128,0.607,6.119402,191.652091,37.280763,#173,2OrWItJ6zAwBNp0OUxK$Bq,1.190365,IfcBeam,0.031930
3,4.300000,0.37800,0.607,6.087885,191.654609,37.280976,#174,2OrWItJ6zAwBNp0OUxK$CR,1.184226,IfcBeam,0.031765
4,0.403000,7.62130,0.503,10.174007,177.246504,37.332488,#175,2OrWItJ6zAwBNp0OUxK_l8,2.142897,IfcBeam,0.057400
5,4.300000,0.37800,0.607,6.087885,191.654609,37.280976,#176,2OrWItJ6zAwBNp0OUxK$Dw,1.184226,IfcBeam,0.031765
6,0.378000,4.32128,0.607,6.119402,191.652091,37.280763,#177,2OrWItJ6zAwBNp0OUxK$Dx,1.190365,IfcBeam,0.031930
7,6.381887,0.37800,0.607,9.171307,191.490330,37.269382,#178,2OrWItJ6zAwBNp0OUxK$8W,1.784993,IfcBeam,0.047894
0,3.908000,17.16600,0.505,131.903519,7.288779,0.283987,#86,1hOSvn6df7F8_7GcBWlRrM,5.139260,IfcSlab,18.096793
1,3.908000,17.16600,0.505,131.712359,7.291085,0.284715,#87,1hOSvn6df7F8_7GcBWlRqU,5.143325,IfcSlab,18.064850
